# Advent of Code 2017
*Phong Nguyen, Oct 2018*

## Lessons Learnt
- Day 1: use `list[-1]` to access the last element of the list: good for handling boundary cases
- Day 3: generator, 2D points and neighbours
- Day 5: in Python we can write 0 < x < 10 instead of an `and`
- Day 7: regular expression `re` is awesome, love `\w` to search for words (excluding any non-alphanumeric letters)

### Some utility functions

In [36]:
import math
import re
from collections import deque, defaultdict, Counter

from numba import jit

def Input(day):
    "Return input file."
    return open('input{}.txt'.format(day))

def InputString(day):
    "Return the content of the input file as a string."
    return Input(day).read()

def InputRows(day):
    "Return the content of the input file as a list of string, each for a row."
    return InputString(day).splitlines()

def InputInts(day):
    "Return the content of the input file as a list of integers, each for a row."
    return [int(x) for x in InputString(day).splitlines()]

def ints(start, end, step=1): return range(start, end+1, step)

# 2D points
UP, LEFT, DOWN, RIGHT = (0, -1), (-1, 0), (0, 1), (1, 0)

def add_tuples(t1, t2):
     return tuple(sum(x) for x in zip(t1, t2))
        
def neighbors8(p): 
    x, y = p
    return ((x-1, y-1), (x, y-1), (x+1, y-1), (x-1, y), (x+1, y), (x-1, y+1), (x, y+1), (x+1, y+1))

def Mht_distance(p):
    return abs(p[0]) + abs(p[1])

def argmax(a):
    return a.index(max(a))

## Day 1: Inverse Captcha
[Problem Description](https://adventofcode.com/2017/day/1)

In [49]:
def day1a(digits):
    return sum(digits[i] 
               for i in range(len(digits)) 
               if digits[i] == digits[i - 1])

digits = [int(d) for d in InputString(1)]    
day1a(digits)

1175

One nice thing here is that `list[-1]` returns the last element of the list. So, no need to handle the circle explicitly.

In [50]:
def day1b(digits):
    n = len(digits)
    return sum(digits[i] 
               for i in range(n) 
               if digits[i] == digits[i - n // 2])
        
day1b(digits)

1166

## Day 2: Corruption Checksum
[Problem Description](https://adventofcode.com/2017/day/2)

In [51]:
def day2a(array):
    return sum(max(nums) - min(nums) for nums in array)

array = [[int(s) for s in row.split()] 
         for row in InputRows(2)]
day2a(array)

42299

In [52]:
def evenly_devide(nums):
    nums = sorted(nums, reverse=True)
    n = len(nums)
    for i in range(n):
        for j in range(i + 1, n):
            if nums[i] % nums[j] == 0:
                return nums[i] // nums[j]

def day2b(array):
    return sum(map(evenly_devide, array))

day2b(array)

277

## Day 3: Spiral Memory
[Problem Description](https://adventofcode.com/2017/day/3)

This is the first challenging question in this year. I will generate the locations of squares following the spiral pattern.

In [86]:
def spiral():
    x, y = 0, 0
    yield x, y
    
    # The spiral goes right N steps, up N steps, then left N+1 steps, down N+1 steps, then right/up N+2, etc.
    N = 0
    while True:
        for dx, dy in (RIGHT, UP, LEFT, DOWN):
            if dx: N += 1
            for _ in range(N):
                x += dx
                y += dy
                yield x, y
                
def day3a(n):
    s = spiral()
    for _ in range(n):
        pos = next(s)
    return Mht_distance(pos)

day3a(347991)

480

In [88]:
def day3b(n):
    values = defaultdict(int)
    values[(0, 0)] = 1
    
    s = spiral()
    pos = next(s) # Skip the first one as already set to 1
    while True:
        pos = next(s)
        values[pos] = sum(values[p] for p in neighbors8(pos))
        if values[pos] > n:
            return values[pos]
    
day3b(347991)

349975

## Day 4: High-Entropy Passphrases
[Problem Description](https://adventofcode.com/2017/day/4)

In [9]:
def is_passphrase(s):
    return len(set(s.split())) == len(s.split())

def day4a(array):
    return sum(map(is_passphrase, array))

day4a(InputRows(4))

455

In [20]:
def is_passphrase2(s):
    words = [''.join(sorted(w)) for w in s.split()]
    return len(set(words)) == len(words)

def day4b(array):
    return sum(map(is_passphrase2, array))

day4b(InputRows(4))

186

## Day 5: A Maze of Twisty Trampolines, All Alike
[Problem Description](https://adventofcode.com/2017/day/5)

In [37]:
def day5a(steps):
    pos = 0
    count = 0
    while (0 <= pos < len(steps)):
        old_pos = pos
        pos += steps[pos]
        steps[old_pos] += 1
        count += 1
        
    return count

day5a(InputInts(5))

342669

In [36]:
@jit
def day5b(steps):
    pos = 0
    count = 0
    N = len(steps)
    while (0 <= pos < N):
        old_pos = pos
        pos += steps[pos]
        offset = (-1 if steps[old_pos] >= 3 else 1)
        steps[old_pos] += offset
        count += 1
        
    return count

day5b(InputInts(5))

25136209

Part 2 is really slow, about 10s. Putting the length of steps outside the long while loop helps reduce to 8.5s. Peter Norvig's code is the same but he used `jit` to run faster. Well, with `jit` (particularly `numba`), it gets to less than half a second!!!

## Day 6: Memory Reallocation
[Problem Description](https://adventofcode.com/2017/day/6)

In [2]:
def redistribute(banks, idx):
    n = banks[idx]
    l = len(banks)
    banks[idx] = 0
    for i in range(1, n + 1):
        banks[(idx + i) % l] += 1
    
def day6a(banks):
    configs = set()
    count = 0
    
    while True:
        t = tuple(banks)
        if t in configs:
            break
            
        configs.add(t)
        redistribute(banks, argmax(banks))
        count += 1

    return count

banks = [int(x) for x in InputString(6).split()]
day6a(banks)

5042

In [3]:
def day6b(banks):
    configs = dict() # save the number of cycles when reaching a config
    count = 0
    
    while True:
        t = tuple(banks)
        if t in configs:
            return count - configs[t]
            
        configs[t] = count
        redistribute(banks, argmax(banks))
        count += 1
    
banks = [int(x) for x in InputString(6).split()]
day6b(banks)

1086

## Day 7: Recursive Circus
[Problem Description](https://adventofcode.com/2017/day/7)

In [37]:
def build_tree(rows):
    def parse(row):
        "Return name, weight, children names"
        name, weight, *rest = re.findall(r'\w+', row)
        return name, int(weight), rest

    node_lookup = defaultdict(dict)
    
    for row in rows:
        name, weight, children = parse(row)
        node = node_lookup[name]
        node['name'] = name
        node['weight'] = node['original_weight']= weight
        node['children'] = []
        for c in children:
            node['children'].append(node_lookup[c])
            node_lookup[c]['parent'] = node       
    
    # Return the root of the tree: the one having children but not parent
    for n in node_lookup.values():
        if 'parent' not in n:
            return n, node_lookup
    
def day7a(rows):
    t, _ = build_tree(rows)
    return t['name']
    
day7a(InputRows(7))

'hmvwl'

In [42]:
def print_tree(node):
    indent = 3
    print(indent * node['depth'] * ' ', node['depth'], node['name'], '(' + str(node['original_weight']) + ')')
    for c in node['children']:
        print_tree(c)
    
def day7b(rows):
    root, node_lookup = build_tree(rows)
    
    # Assign depths: root = 0
    def assign_depth(node):
        for c in node['children']:
            c['depth'] = node['depth'] + 1
            assign_depth(c)
    
    root['depth'] = 0
    assign_depth(root)

    max_depth = max(n['depth'] for n in node_lookup.values())

    # Checking weight from the leaf
    for level in range(max_depth - 1, -1, -1):
        nodes = [n for n in node_lookup.values() if n['depth'] == level]
        for n in nodes:
            if n['children']:
                weights = [c['weight'] for c in n['children']]
                if len(set(weights)) == 1:
                    n['weight'] += sum(weights)
                else:
                    correct_weight = Counter(weights).most_common(1)[0][0]
                    wrong_weight = Counter(weights).most_common()[-1][0]
                    diff = correct_weight - wrong_weight
                    wrong_node = next(x for x in n['children'] if x['weight'] == wrong_weight)
                    return wrong_node['original_weight'] + diff

day7b(InputRows(7))

1853

Took long time for me to do the second part because I assume that the tree is balance, i.e., all leaf nodes have the same height.